<a href="https://colab.research.google.com/github/ireneraven/IDRiD/blob/master/U_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nueva sección

/content/train

# Modelo de U-Net para procesado masivo de datos

In [0]:
#Importar los módulos
import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as mtpimg
import sys
import os
from pathlib import Path
from skimage import io
from PIL import Image
from tqdm import tqdm

from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

from tensorflow.compat.v1 import RunOptions
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Lambda
from keras.layers.core import Lambda, RepeatVector, Reshape
from tensorflow.keras.layers  import Conv2D, Conv2DTranspose
from tensorflow.keras.layers  import MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers  import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


Using TensorFlow backend.


Ahora a partir de los directorios vamos a coger las imágenes que tenemos de train y sus correspondientes máscaras:

In [0]:
def get_data(path, path2):
    
    #otra manera sería: filesInPathTrain = next(os.walk(myPath))[2] - Estaría ya ordenado
    
    filesInPathTrain = os.listdir(path)
    filesInPathTrain.sort(key = len)
    imagesTrain = np.zeros((len(filesInPathTrain), 2848, 4288, 3), dtype=np.float32)
    
    filesInPathMask = os.listdir(path2)
    filesInPathMask.sort(key = len)
    imagesMask = np.zeros((len(filesInPathMask), 2848, 4288, 3), dtype=np.float32)
    

    for i, filename in tqdm(enumerate(filesInPathTrain)):
        imageToRead = path + r'\\' + str(filename)
        img = img_to_array(mtpimg.imread(imageToRead))/255.0
        if img is not None:
            imagesTrain[i] = img
        

        imageToRead2 = path2 + r'\\' + str(filesInPathMask[i])
        img2 = img_to_array(io.imread(imageToRead2))/255.0
        if img2 is not None:
            imagesMask[i] = img2
        


    return imagesTrain, imagesMask

Definimos la U-Net
![texto alternativo](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png)


In [0]:
# 1) Since each block consicts on two convolutions:

def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


# 2) We create a function to define the UNET Model
def get_unet(inputs, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(inputs, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(3, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model


El código que nos ha dicho JL. que puede resolver el problema (a mi no me iba)

In [0]:
# Specifying the input shape
input_shape = Input((2848, 4288, 3), dtype = 'float32')

#For the OOM error
run_opts = RunOptions(report_tensor_allocations_upon_oom = True)

model = get_unet(input_shape, n_filters=16, dropout=0.1, batchnorm=True)
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])#, options = run_opts)
model.summary()
#Cuando sean más cambiar a categorical_crossentropy

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2848, 4288,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 2848, 4288, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 2848, 4288, 1 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 2848, 4288, 1 0           batch_normalization[0][0]        
______________________________________________________________________________________________

Ahora usamos la función para coger los datos y luego dividir las imágenes y sus máscaras en train y valid

In [0]:
myPath = r"C:\\Users\\pablo\\Documents\\2_PABLO\\2. Universidad\\(4) Cuarto año\\2_Segundo semestre\\Procesado masivo de datos\\Programación\\Fotos\\A. Segmentation\\1. Original Images\\a. Training Set"
myMaskPath = r"C:\\Users\\pablo\\Documents\\2_PABLO\\2. Universidad\\(4) Cuarto año\\2_Segundo semestre\\Procesado masivo de datos\\Programación\\Fotos\\A. Segmentation\\2. All Segmentation Groundtruths\\a. Training Set\5. Optic Disc"

X, y = get_data(myPath, myMaskPath)

#Dividir en X_train, X_valid, y_train, y_valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=2018)


FileNotFoundError: ignored

Ahora ponemos el modelo a entrenarse

In [0]:
#Estas son las callbacks, las definimos para que no se produzca overfitting y underfitting y además se guarde mi modelo.
modelCallbacks = [
    EarlyStopping(monitor = 'val_loss', patience=10, verbose=1),
    ReduceLROnPlateau(monitor = 'val_loss', factor=0.2, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-opticDisc.h5', verbose=1, save_best_only=True, save_weights_only=True),
    TensorBoard(log_dir = 'logs')
    ]

#Esta línea antes daba error
tf.compat.v1.disable_eager_execution()

results = model.fit(X_train, y_train, batch_size=None, epochs=100, verbose = 2, callbacks=modelCallbacks, validation_data=(X_valid, y_valid))

